<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/04_Extensiones_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1)  Instalación y dependencias

In [1]:
#!pip install numpy==1.26.4 tomotopy==0.13.0 pandas==2.2.2
!pip install --upgrade tomotopy

import tomotopy as tp
import pandas as pd
import numpy as np
from datetime import datetime

#2) Labeled LDA

In [2]:
# Crear modelo LLDA
llda = tp.PLDAModel()

# Datos de ejemplo con sus etiquetas
documentos = [
    (['tecnología'], 'Los nuevos avances en inteligencia artificial permiten procesar datos más rápido'),
    (['ciencia'], 'Los científicos descubren un nuevo planeta similar a la Tierra'),
    (['medicina'], 'Un nuevo tratamiento promete curar enfermedades cardíacas'),
    (['tecnología', 'ciencia'], 'Los supercomputadores ayudan a simular fenómenos físicos complejos')
]

# Añadir documentos al modelo
for labels, texto in documentos:
    llda.add_doc(words=texto.split(), labels=labels)

# Entrenar modelo
llda.train(10)

# Para cada tópico, encontrar su etiqueta más común
etiquetas_por_topico = {}
for doc_idx, (labels, _) in enumerate(documentos):
    doc = llda.docs[doc_idx]
    topico_principal = np.argmax(doc.get_topic_dist())
    if topico_principal not in etiquetas_por_topico:
        etiquetas_por_topico[topico_principal] = []
    etiquetas_por_topico[topico_principal].extend(labels)

# Mostrar tópicos con sus etiquetas y palabras
print("\n🔍 Tópicos encontrados:")
for topico in range(llda.k):
    # Encontrar la etiqueta más común para este tópico
    if topico in etiquetas_por_topico:
        etiquetas = etiquetas_por_topico[topico]
        etiqueta_mas_comun = max(set(etiquetas), key=etiquetas.count)
    else:
        etiqueta_mas_comun = "desconocida"

    print(f"\n📌 Tópico {topico + 1} (Etiqueta: {etiqueta_mas_comun})")

    # Obtener palabras y sus pesos
    for palabra, peso in llda.get_topic_words(topico, top_n=10):
        barra = "█" * int(peso * 50)  # Visualización del peso
        print(f"{palabra}: {peso:.4f} {barra}")


🔍 Tópicos encontrados:

📌 Tópico 1 (Etiqueta: tecnología)
permiten: 0.0892 ████
Los: 0.0892 ████
rápido: 0.0892 ████
más: 0.0892 ████
datos: 0.0892 ████
procesar: 0.0892 ████
artificial: 0.0892 ████
inteligencia: 0.0892 ████
en: 0.0892 ████
avances: 0.0892 ████

📌 Tópico 2 (Etiqueta: ciencia)
a: 0.1097 █████
Los: 0.1097 █████
planeta: 0.0551 ██
complejos: 0.0551 ██
físicos: 0.0551 ██
fenómenos: 0.0551 ██
simular: 0.0551 ██
ayudan: 0.0551 ██
supercomputadores: 0.0551 ██
Tierra: 0.0551 ██

📌 Tópico 3 (Etiqueta: medicina)
nuevo: 0.1380 ██████
Un: 0.1380 ██████
tratamiento: 0.1380 ██████
promete: 0.1380 ██████
curar: 0.1380 ██████
enfermedades: 0.1380 ██████
cardíacas: 0.1380 ██████
supercomputadores: 0.0014 
ayudan: 0.0014 
simular: 0.0014 


#3) Dynamic Topic Model

In [3]:
import tomotopy as tp
from datetime import datetime

# Crear modelo dinámico (2 tópicos, 3 periodos de tiempo)
dtm = tp.DTModel(k=2, t=3)  # k tópicos, 3 periodos de tiempo

# Datos de ejemplo a través del tiempo
documentos_tiempo = [
    # 2020
    (2020, 'La pandemia global afecta a millones de personas en todo el mundo'),
    (2020, 'Los científicos trabajan en el desarrollo de vacunas'),
    # 2021
    (2021, 'Las campañas de vacunación avanzan en diferentes países'),
    (2021, 'Nuevas variantes del virus preocupan a los expertos'),
    # 2022
    (2022, 'La población mundial alcanza altos niveles de inmunidad'),
    (2022, 'Los países comienzan a levantar restricciones sanitarias')
]

# Añadir documentos
for año, texto in documentos_tiempo:
    # Calcular el timestep (0 para 2020, 1 para 2021, 2 para 2022)
    timestep = año - 2020
    # Añadir documento con su timestep correspondiente
    dtm.add_doc(words=texto.split(), timepoint=timestep)

# Entrenar modelo
dtm.train(10)

# Mostrar evolución de tópicos
print("\n Evolución de tópicos:\n")
for topic in range(dtm.k):
    print(f"\n Tópico {topic + 1}:")
    for t in range(dtm.num_timepoints):
        año = 2020 + t
        print(f"\n {año}:")
        # Mostrar palabras y sus pesos
        for palabra, peso in dtm.get_topic_words(topic, timepoint=t, top_n=5):
            barra = "█" * int(peso * 50)
            print(f"{palabra}: {peso:.4f} {barra}")


 Evolución de tópicos:


 Tópico 1:

 2020:
de: 0.1377 ██████
La: 0.0754 ███
vacunas: 0.0753 ███
todo: 0.0752 ███
millones: 0.0750 ███

 2021:
países: 0.0545 ██
variantes: 0.0542 ██
Las: 0.0542 ██
de: 0.0542 ██
diferentes: 0.0542 ██

 2022:
comienzan: 0.0683 ███
sanitarias: 0.0683 ███
mundial: 0.0683 ███
de: 0.0682 ███
inmunidad: 0.0682 ███

 Tópico 2:

 2020:
el: 0.0958 ████
mundo: 0.0544 ██
trabajan: 0.0544 ██
Los: 0.0543 ██
afecta: 0.0543 ██

 2021:
Nuevas: 0.1379 ██████
del: 0.1377 ██████
campañas: 0.1376 ██████
avanzan: 0.1376 ██████
de: 0.0129 

 2022:
a: 0.0964 ████
restricciones: 0.0960 ████
levantar: 0.0960 ████
población: 0.0960 ████
La: 0.0958 ████


#4) Nuevo Documento

In [4]:
# Primero creamos un mapeo de tópicos a etiquetas basado en el entrenamiento
def crear_mapeo_topicos_etiquetas(modelo, documentos_entrenamiento):
    topico_a_etiqueta = {}
    for doc_idx, (etiquetas, _) in enumerate(documentos_entrenamiento):
        doc = modelo.docs[doc_idx]
        # Obtener el tópico dominante para este documento
        topico_principal = np.argmax(doc.get_topic_dist())
        if topico_principal not in topico_a_etiqueta:
            topico_a_etiqueta[topico_principal] = etiquetas[0]  # Tomamos la primera etiqueta
    return topico_a_etiqueta

# Función  para analizar documentos
def analizar_documento(texto, modelo, mapeo_topicos):
    # Preprocesar el texto
    palabras = texto.split()

    # Inferir tópicos
    doc = modelo.make_doc(palabras)
    topicos = modelo.infer(doc)[0]

    print(f"\nTexto analizado: '{texto}'")
    print("\nDistribución de tópicos:")

    # Mostrar distribución para cada tópico
    for topico_idx, prob in enumerate(topicos):
        etiqueta = mapeo_topicos.get(topico_idx, f"Tópico {topico_idx+1}")
        barra = "▓" * int(prob * 50)
        print(f"{etiqueta}: {prob:.2%} {barra}")
        # Mostrar palabras más relevantes del tópico
        palabras = [word for word, _ in modelo.get_topic_words(topico_idx, top_n=3)]
        print(f"Palabras clave: {', '.join(palabras)}\n")

# Crear el mapeo de tópicos a etiquetas
mapeo_topicos = crear_mapeo_topicos_etiquetas(llda, documentos)

# Ejemplo de uso
nuevo_texto = "La inteligencia artificial ayuda a los científicos a desarrollar nuevos medicamentos para curar enfermedades"
analizar_documento(nuevo_texto, llda, mapeo_topicos)


Texto analizado: 'La inteligencia artificial ayuda a los científicos a desarrollar nuevos medicamentos para curar enfermedades'

Distribución de tópicos:
tecnología: 37.40% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Palabras clave: permiten, Los, rápido

ciencia: 37.60% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Palabras clave: a, Los, planeta

medicina: 25.00% ▓▓▓▓▓▓▓▓▓▓▓▓
Palabras clave: nuevo, Un, tratamiento

